In [ ]:
# === Imports ===

# Qiskit and maths
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import Initialize, QFT, TwoLocal
from qiskit_aer.primitives import Sampler

# Torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# Finance information
import yfinance as yf
import pandas as pd

# Extras :)
from tqdm import tqdm
from functools import partial
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
class qGAN:
    """
        This class implements the Quantum Generative Adversarial Network based on the paper in nature
        ```Quantum Generative Adversarial Networks for learning and loading random distributions``` 
        from Zoufal, Lucchi, and Woerner (https://www.nature.com/articles/s41534-019-0223-2#ref-CR32).

        The techniques this class implements that the paper either doesn't mention or doesn't include
        are: TwoLocal circuit preparation, prioritising generator training (roughly twice as often as 
        the discriminator), label smoothing, explicit binary cross entropy with real and fake loss for
        the discriminator, gradient clipping for the disc and genr, both KL and Entropy regularisation
        to optimise for quantum states (still helpful for classical problems). We also do some 
        post-calculations including, KL divergence, Entropy, Coverage, and Loss with graphs.
    """

    def __init__(
            self, model, samples, rotation_gate='ry', entangle_gate='cz', epochs=100, num_samples=1000, n_qubits=6, k_layers=3,
            batch_size=32, gen_lr=2e-4, disc_lr=1e-4, bounds=[-1.5, 1.5], sampler=Sampler(), device='cpu', circuit=None,
                ):
        # Problem specific variables
        self.num_samples = num_samples # Ideally around 1000 samples or more
        self.n_qubits = n_qubits       # A good range is 4 - 8 qubits
        self.k_layers = k_layers       # Best around 2 - 3 repititions
        self.model = model             # torch.nn.Sequential: linear, leakyReLU, linear, Sigmoid (Might be improved in the future)
        self.samples = samples         # A torch tensor float 32 that matches the target distribution
        self.batch_size = batch_size   # Batching data into roughly 32, optimises runtime
        self.gen_lr = gen_lr           # Keep this a bit higher than the disc_lr
        self.disc_lr = disc_lr         # Around 5e-5 to 1e-4, this should not overpower the generator
        self.bounds = bounds           # Problem specific, but this is just generally for the QFT state prep. problem
        self.epochs = epochs           # Best to have around 300, if you have 100 a quantum state will not be properly learnt
        self.sampler = sampler         # Base use is the qiskit simulator sampler
        self.device = device           # Option of cpu or gpu
        self.func = circuit            # In case people want to build something other than a TwoLocal circuit
        # Ansatz specific gates
        self.rot_gate = rotation_gate
        self.ent_gate = entangle_gate

        # Standard/Computed Variables
        self.epsilon = 1e-8
        bins = 2 ** self.n_qubits
        self.QC_TEMPLATE, self.NOISE_PARAMS, self.ANSATZ_PARAMS = None, None, None # build_parameterized_circuit in model initialisation
        self.dummy_ansatz = None # Basic TwoLocal for this case based on rotation gates
        self.num_ansatz_params = None # Based on dummy ansatz params
        self.theta = None # Defined in the initialisation
        self.num_params = None # Defined in the intialisation
        self.dataloader = DataLoader(TensorDataset(self.samples), batch_size=32, shuffle=True)

        self.initialise_model() # Fixing the None variables

    
    def build_parameterised_circuit(self, build_func=None):
        """
            This function builds the parameterised circuit with either TwoLocal or the users choice.
        """
        noise_params = ParameterVector("ζ", length=self.n_qubits)
        
        ansatz = build_func if build_func else TwoLocal(
            self.n_qubits, self.rot_gate, self.ent_gate,
            reps=self.k_layers, entanglement='circular', parameter_prefix="θ"
        )
        
        qc = QuantumCircuit(self.n_qubits)
        
        for i in range(self.n_qubits):
            qc.ry(noise_params[i], i)
        
        qc.compose(ansatz, inplace=True)
        qc.measure_all()
        
        return qc, list(noise_params), list(ansatz.parameters)

    
    def generator_update(self, noise_batch):
        """
            This is a function that updates the generator based on the noise, with clipping.
        """
        shift = np.pi / 2
        grad = torch.zeros_like(self.theta)
        batch_size = noise_batch.size(0)
        ones = torch.ones((batch_size, 1)).to(self.device)
    
        with torch.no_grad():
            for i in range(self.num_params):
                theta_plus = self.theta.clone()
                theta_plus[i] += shift
                theta_minus = self.theta.clone()
                theta_minus[i] -= shift
    
                g_plus = self.quantum_generator(noise_batch, theta_plus)
                g_minus = self.quantum_generator(noise_batch, theta_minus)
    
                D_plus = self.model(g_plus)
                D_minus = self.model(g_minus)
    
                loss_plus = F.binary_cross_entropy(D_plus, ones)
                loss_minus = F.binary_cross_entropy(D_minus, ones)
    
                grad[i] = ((loss_plus - loss_minus) / 2).clamp(-1.0, 1.0)
    
            self.theta -= self.gen_lr * grad

                         
    def build_func(self):
        '''
            Just a short function in case peopole want to use something other than TwoLocal.
            Returns a the circuit with the rotation and entanglements.
        '''
        if not self.func:
            return TwoLocal(self.n_qubits, self.rot_gate, self.ent_gate, reps=self.k_layers, entanglement='circular')
        return func(self.n_qubits, self.rot_gate, self.ent_gate, self.k_layers)


    def quantum_generator(self, noise_batch, theta):
        '''
            This is the main portion of our qGAN model which runs the sampler on the quantum
            circuit to get batch_outputs for results.
        '''
        batch_size = noise_batch.shape[0]
        flat_theta = theta.detach().cpu().numpy()
    
        noise_np = noise_batch.detach().cpu().numpy()
        lower, upper = self.bounds
        noise_angles = (noise_np + 1) / 2 * (upper - lower) + lower
    
        parameter_values = []
        for i in range(batch_size):
            noise_vec = noise_angles[i][:len(self.NOISE_PARAMS)]
            theta_vec = flat_theta[:len(self.ANSATZ_PARAMS)]
            parameter_values.append(list(noise_vec) + list(theta_vec))
    
        results = self.sampler.run([self.QC_TEMPLATE] * batch_size, parameter_values=parameter_values, shots=2048).result()
        batch_outputs = []
        dim = 2 ** self.n_qubits
        for dist in results.quasi_dists:
            hist = np.zeros(dim)
            for bitstring, prob in dist.items():
                idx = int(bitstring, 2) if isinstance(bitstring, str) else int(bitstring)
                hist[idx] = prob
            batch_outputs.append(hist)
    
        batch_outputs = np.stack(batch_outputs, axis=0).astype(np.float32)
        return torch.tensor(batch_outputs, dtype=torch.float32, device=self.device)
        
    
    def initialise_model(self):
        # Necessary params
        self.QC_TEMPLATE, self.NOISE_PARAMS, self.ANSATZ_PARAMS = self.build_parameterised_circuit()
        self.dummy_ansatz = self.build_func()
        self.num_ansatz_params = len(self.dummy_ansatz.parameters)
        self.theta = nn.Parameter(torch.rand(len(self.ANSATZ_PARAMS), dtype=torch.float32, requires_grad=True).to(self.device))
        self.num_params = len(self.theta)

    def train(self):
        ######## MAKE SURE QUANTUM_GENERATOR FUNCTIONS TAKE NEW INPUTS
        '''
        This is the function to train the model.
        '''
        gen_losses, disc_losses, kl_divs, entropies, mode_coverages = [], [], [], [], []

        self.model.to(self.device)
        bins = 2 ** self.n_qubits
        dis_opt = optim.Adam(self.model.parameters(), lr=self.disc_lr, amsgrad=True)
    
        for epoch in range(self.epochs):
            for real_data_batch in tqdm(self.dataloader, desc=f"Epoch {epoch+1}"):
                real_data = real_data_batch[0].to(self.device)
                batch_size = real_data.size(0)
    
                # Train Discriminator
                dis_opt.zero_grad()
                real_loss = F.binary_cross_entropy(self.model(real_data), torch.full((batch_size,1),0.8).to(self.device))
    
                noise = torch.randn(batch_size, self.n_qubits).to(self.device)
                fake_data = self.quantum_generator(noise, self.theta).detach()
                fake_loss = F.binary_cross_entropy(self.model(fake_data), torch.full((batch_size,1),0.2).to(self.device))
    
                disc_loss = real_loss + fake_loss
                disc_loss.backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                dis_opt.step()
    
                # Generator updates (twice as frequent)
                for _ in range(2):
                    noise = torch.randn(batch_size, self.n_qubits).to(self.device)
                    fake_preds = self.model(self.quantum_generator(noise, self.theta))
                    gen_loss = F.binary_cross_entropy(fake_preds, torch.full((batch_size,1),0.9).to(self.device))
    
                    with torch.no_grad():
                        real_dist = real_data.mean(dim=0) + self.epsilon
                        gen_dist = fake_data.mean(dim=0) + self.epsilon
                        kl_reg = F.kl_div((gen_dist/gen_dist.sum()).log(), real_dist/real_dist.sum(), reduction='batchmean')
                        entropy = -(gen_dist/gen_dist.sum() * (gen_dist/gen_dist.sum()).log()).sum()
    
                    gen_loss += 0.25 * kl_reg - 0.01 * entropy
                    self.generator_update(noise)
    
            gen_losses.append(gen_loss.item())
            disc_losses.append(disc_loss.item())

    
            # === KL Divergence, Entropy, Mode Coverage ===
            noise = torch.randn(1000, self.n_qubits).to(self.device)
            fake_samples = self.quantum_generator(noise, self.theta).detach().to(self.device)
            fake_flat = fake_samples.mean(dim=0)
            real_flat = real_data[:len(fake_samples)].mean(dim=0).to(self.device)
    
            fake_prob = fake_flat / fake_flat.sum() + self.epsilon
            real_prob = real_flat / real_flat.sum() + self.epsilon
    
            kl = F.kl_div(fake_prob.log(), real_prob, reduction='batchmean')
            entropy = -torch.sum(fake_prob * fake_prob.log()).item()
            mode_coverage = (fake_prob > 1e-3).sum().item()
    
            kl_divs.append(kl.item())
            entropies.append(entropy)
            mode_coverages.append(mode_coverage)
    
            # === Combined LIVE PLOTS ===
            clear_output(wait=True)
            fig, axs = plt.subplots(1, 3, figsize=(18, 4))
    
            axs[0].plot(gen_losses, label="Generator")
            axs[0].plot(disc_losses, label="Discriminator")
            axs[0].set_title("Losses")
            axs[0].legend()
            axs[0].grid(True)
    
            axs[1].plot(kl_divs, label="KL(G || R)", color='purple')
            axs[1].set_title("KL Divergence")
            axs[1].legend()
            axs[1].grid(True)
    
            real_indices = np.random.choice(np.arange(bins), p=real_prob.numpy(), size=1000)
            fake_indices = np.random.choice(np.arange(bins), p=fake_prob.numpy(), size=1000)
            axs[2].hist(real_indices, bins=bins, alpha=0.5, label="Real", density=True)
            axs[2].hist(fake_indices, bins=bins, alpha=0.5, label="Generated", density=True)
            axs[2].set_title("Distribution Comparison")
            axs[2].legend()
            axs[2].grid(True)
    
            plt.tight_layout()
            plt.show()
    
            print(f"Epoch [{epoch+1}/{self.epochs}]  D_loss: {disc_loss.item():.4f}  G_loss: {gen_loss.item():.4f}  KL: {kl.item():.4f}  Entropy: {entropy:.4f}  Coverage: {mode_coverage}/{bins}")


In [ ]:
# === Create QFT States ===
# Create QFT states
n_qubits = 6

# Initialize basis state |x⟩ = |5⟩ for example (can be any 0–7)
x = 5
qc = QuantumCircuit(n_qubits)
qc.initialize([0]*x + [1] + [0]*(2**n_qubits - x - 1), range(n_qubits))

# Apply QFT
qft = QFT(num_qubits=n_qubits, do_swaps=False).decompose()
qc.append(qft, range(n_qubits))

# Simulate
sv = Statevector.from_instruction(qc)

# Measurement probabilities
probs = sv.probabilities_dict()

# Turn into vector over 2^n bins
target_distribution = np.zeros(2**n_qubits)
for bitstring, prob in probs.items():
    idx = int(bitstring, 2)
    target_distribution[idx] = prob

# Convert to torch tensor
real_data = torch.tensor(target_distribution, dtype=torch.float32)

In [ ]:
# Prepare samples
num_samples = 1000
samples_np = np.tile(target_distribution, (num_samples, 1))
samples = torch.tensor(samples_np, dtype=torch.float32)

# Create discriminator model
model = nn.Sequential(
    nn.Linear(2**6, 20),
    nn.LeakyReLU(0.01),
    nn.Linear(20, 1),
    nn.Sigmoid()
)

In [ ]:
# Run model
qgan = qGAN(model=model, samples=samples)
qgan.train()